# Taller 1

## 0. Preparación inicial

### 0.1. Importación de librerías y configuraciones

In [2]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Pandas configuration for extending the number of columns and rows to show
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### 0.2. Carga de datos

In [16]:
# Loading the CSV file as dataframe
hotels_df = pd.read_csv("./data/hotel_bookings_modified.csv")
hotels_df_dict = pd.read_excel("./data/Hotel Bookings Demand Data Dictionary.xlsx")

C:\Users\user\AppData\Local\Temp\ipykernel_62188\2951495078.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  hotels_df = pd.read_csv("./data/hotel_bookings_modified.csv")


## 1. Entendimiento de los datos

Como primer paso es necesario revisar el tipo de variable de cada característica, así como el número de reservas con los que se cuenta.

In [5]:
# Showing dataframe dimensions
hotels_df.shape

(58895, 33)

| #  | Columna                       | Tipo      | Descripción | Clasificación |
|----|-------------------------------|-----------|-------------|---------------|
| 1  | hotel                         | text      | Hotel (H1 = Resort Hotel o H2 = City Hotel) | Categórica |
| 2  | is_canceled                   | boolean   | Indica si la reserva fue cancelada (1) o no (0) | Categórica |
| 3  | lead_time                     | integer   | Días entre la fecha en que se realizó la reserva en el PMS y la fecha de llegada | Numérica |
| 4  | arrival_date_year             | integer   | Año de la fecha de llegada | Numérica |
| 5  | arrival_date_month            | text      | Mes de la fecha de llegada | Categórica |
| 6  | arrival_date_week_number      | integer   | Número de semana del año de la fecha de llegada | Numérica |
| 7  | arrival_date_day_of_month     | integer   | Día del mes de la fecha de llegada | Numérica |
| 8  | stays_in_weekend_nights       | integer   | Noches de fin de semana (sábado o domingo) reservadas o utilizadas | Numérica |
| 9  | stays_in_week_nights          | integer   | Noches de semana (lunes a viernes) reservadas o utilizadas | Numérica |
| 10 | adults                        | integer   | Número de adultos | Numérica |
| 11 | children                      | integer   | Número de niños | Numérica |
| 12 | babies                        | integer   | Número de bebés | Numérica |
| 13 | meal                          | text      | Tipo de plan de comidas reservado | Categórica |
| 14 | country                       | text      | País de origen en formato ISO 3155–3:2013 | Categórica |
| 15 | market_segment                | text      | Segmento de mercado (TA = Travel Agents, TO = Tour Operators) | Categórica |
| 16 | distribution_channel          | text      | Canal de distribución (TA = Travel Agents, TO = Tour Operators) | Categórica |
| 17 | is_repeated_guest             | boolean   | Indica si es un cliente repetido (1) o no (0) | Categórica |
| 18 | previous_cancellations        | integer   | Número de reservas previas canceladas por el cliente | Numérica |
| 19 | previous_bookings_not_canceled| integer   | Número de reservas previas no canceladas por el cliente | Numérica |
| 20 | reserved_room_type            | text      | Código del tipo de habitación reservada | Categórica |
| 21 | assigned_room_type            | text      | Código del tipo de habitación asignada (puede diferir de la reservada) | Categórica |
| 22 | booking_changes               | integer   | Número de cambios realizados a la reserva | Numérica |
| 23 | deposit_type                  | text      | Tipo de depósito | Categórica |
| 24 | agent                         | integer   | ID de la agencia de viajes que hizo la reserva | Numérica (ID) |
| 25 | company                       | integer   | ID de la compañía/entidad responsable de la reserva | Numérica (ID) |
| 26 | days_in_waiting_list          | integer   | Días en lista de espera antes de la confirmación | Numérica |
| 27 | customer_type                 | text      | Tipo de cliente | Categórica |
| 28 | adr                           | numeric   | Tarifa diaria promedio (ADR) | Numérica |
| 29 | required_car_parking_spaces   | integer   | Número de espacios de parqueadero requeridos | Numérica |
| 30 | total_of_special_requests     | integer   | Número de solicitudes especiales del cliente | Numérica |
| 31 | reservation_status            | text      | Estado final de la reserva | Categórica |
| 32 | reservation_status_date       | date      | Fecha en la que se estableció el último estado de la reserva | Fecha |


Gracias al diccionario, es posible identificar las variables categóricas y numéricas del dataset.

In [18]:
# Showing column types
hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58895 entries, 0 to 58894
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           58895 non-null  object 
 1   is_canceled                     58895 non-null  int64  
 2   lead_time                       58895 non-null  int64  
 3   arrival_date_year               58307 non-null  float64
 4   arrival_date_month              58895 non-null  object 
 5   arrival_date_week_number        58895 non-null  int64  
 6   arrival_date_day_of_month       58895 non-null  int64  
 7   stays_in_weekend_nights         58895 non-null  int64  
 8   stays_in_week_nights            58895 non-null  int64  
 9   adults                          58895 non-null  int64  
 10  children                        58891 non-null  float64
 11  babies                          58895 non-null  int64  
 12  meal                            

Gracias a esta pequeña comparación ya fue posible identificar columnas que no existen en el diccionario de los datos, por ejemplo la columna `kids` no aparece en el diccionario. Sin embargo, sí existen otras dos columnas con el mismo propósito las cuales son `children` `babies`. Esto se tendrá en cuenta más adelante.

In [7]:
hotels_df.sample(5) # Showing a sample of n rows

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,kids
31018,Resort Hotel,0,2,2017.0,January,4,28,0,1,2,0.0,0,BB,PRT,Online TA,TA/TO,1,0,1,D,D,0,No Deposit,240.0,NaN,0.0,Transient,66.0,0.0,1.0,Check-Out,2017-01-29,NaN
38228,Resort Hotel,0,3,2017.0,July,28,10,1,1,2,1.0,0,HB,PRT,Online TA,TA/TO,0,0,0,C,C,1,No Deposit,240.0,NaN,0.0,Transient,193.0,0.0,1.0,Check-Out,2017-07-12,NaN
12772,Resort Hotel,1,333,2017.0,July,29,16,2,4,2,0.0,0,BB,ESP,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240.0,NaN,0.0,Transient,152.1,0.0,4.0,Canceled,2017-05-16,NaN
17177,Resort Hotel,0,0,2015.0,September,40,28,1,0,2,1.0,0,BB,AUS,Direct,Direct,0,0,0,A,G,0,No Deposit,NaN,NaN,0.0,Transient,131.0,1.0,1.0,Check-Out,2015-09-29,NaN
28092,Resort Hotel,0,5,2016.0,September,39,19,1,0,1,0.0,0,BB,ESP,Online TA,TA/TO,0,0,0,A,C,0,No Deposit,240.0,NaN,0.0,Transient-Party,86.0,0.0,2.0,Check-Out,2016-09-20,NaN


In [19]:
hotels_df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,kids
count,58895.000000,58895.000000,58307.000000,58895.000000,58895.000000,58895.000000,58895.000000,58895.000000,58891.000000,58895.000000,58895.000000,58895.000000,58895.000000,58895.000000,49763.000000,58894.000000,58894.000000,58306.000000,58894.000000,201.000000
mean,0.411393,100.050904,2205.522939,27.837389,15.766432,1.058154,2.847576,1.969148,0.112937,0.136599,0.030189,0.069191,0.099618,0.243009,146.983080,3.525979,96.250426,0.096697,0.512225,34.417910
std,0.492090,101.162420,1837.392466,13.346053,8.783036,1.093032,2.235486,2.943454,0.419710,3.115889,0.171109,1.102138,0.829166,0.695205,120.114987,21.841676,58.555599,0.300382,0.768378,38.571032
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-6.380000,0.000000,0.000000,-1.000000
25%,0.000000,17.000000,2016.000000,17.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,60.000000,0.000000,0.000000,-1.000000
50%,0.000000,69.000000,2016.000000,29.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,196.000000,0.000000,84.000000,0.000000,0.000000,-1.000000
75%,1.000000,157.000000,2016.000000,38.000000,23.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,240.000000,0.000000,120.000000,0.000000,1.000000,65.000000
max,1.000000,737.000000,20016.000000,53.000000,31.000000,19.000000,50.000000,100.000000,10.000000,100.000000,1.000000,26.000000,30.000000,20.000000,535.000000,391.000000,5400.000000,8.000000,5.000000,100.000000


| Columna                      | Problema principal                                                                 | Evidencia (describe / info)                                 |
|------------------------------|-----------------------------------------------------------------------------------|-------------------------------------------------------------|
| arrival_date_year            | Valores fuera de rango (typos: ej. `20016`) que distorsionan análisis temporales  | `min=2015`, `25%=2016`, `max=20016`, `mean≈2205`            |
| adults                       | Valores negativos (-1) en un conteo que no puede ser negativo                     | `min = -1`                                                  |
| children                     | Valores negativos (-1) y tipo float por algunos nulos                              | `min = -1`, `count=58891`                                   |
| adr                          | Tarifas negativas (no tiene sentido económico)                                    | `min = -6.38`                                               | 
| stays_in_week_nights         | Valores máximos muy altos (posibles outliers o errores de entrada)                | `max = 50`                                                  |
| stays_in_weekend_nights      | Valores máximos inusuales (posible outlier)                                       | `max = 19`                                                  |
| agent                        | Muchos nulos (posible ausencia de agencia)                                        | `count=49763`                                               |


Gracias a la tabla de descripción de los datos es posible identificar las diferentes inconsistencias que existen dentro del conjunto de datos.

In [27]:
print("Total de registros duplicados iniciales: ",hotels_df.duplicated(keep = False).sum())
print("Porcentaje de registros duplicados ",round((hotels_df.duplicated(keep = False).sum()/hotels_df.shape[0])*100,2), "%")

Total de registros duplicados iniciales:  17159
Porcentaje de registros duplicados  29.13 %


Hay una cantidad considerable de registros duplicados, esto es importante tenerlo en cuenta porque significa que no hay tantos registros útiles como se pensaba inicialmente.

In [11]:
hotels_df.shape

(58895, 33)

In [23]:
round(100*(hotels_df.isnull().sum()/len(hotels_df)), 2).sort_values(ascending=False)

kids                              99.66
company                           94.09
agent                             15.51
arrival_date_year                  1.00
reservation_status                 1.00
required_car_parking_spaces        1.00
customer_type                      1.00
country                            0.81
children                           0.01
assigned_room_type                 0.00
booking_changes                    0.00
deposit_type                       0.00
days_in_waiting_list               0.00
previous_bookings_not_canceled     0.00
adr                                0.00
total_of_special_requests          0.00
reservation_status_date            0.00
reserved_room_type                 0.00
hotel                              0.00
previous_cancellations             0.00
is_canceled                        0.00
distribution_channel               0.00
market_segment                     0.00
meal                               0.00
babies                             0.00
